# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [2]:
conn = sqlite3.connect("/content/Db-IMDB-Assignment.db")

#### Overview of all tables

In [3]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [4]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [5]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """

SELECT   
          Person.Name AS director_name,
          Movie.title AS movie_title,
          CAST(SUBSTR(TRIM(Movie.year),-4,4) AS INT) AS release_year
  FROM   Person 
  JOIN   M_Director
    ON   Person.PID = M_Director.PID
  JOIN   Movie
    ON   Movie.MID = M_Director.MID
  JOIN   M_Genre
    ON   M_Genre.MID = Movie.MID
  JOIN   Genre
    ON   M_Genre.GID = Genre.GID
  WHERE 
          Genre.Name LIKE '%Comedy%' AND
          ((release_year%4 = 0 AND release_year%100 !=0) OR
          (release_year%4 = 0 AND release_year%100 = 0 AND release_year%400 = 0))  
          
ORDER BY release_year DESC;
         
"""
grader_1(query1)

          director_name           movie_title  release_year
0          Milap Zaveri            Mastizaade          2016
1          Shakun Batra         Kapoor & Sons          2016
2          Rohit Dhawan               Dishoom          2016
3           Nitya Mehra       Baar Baar Dekho          2016
4         Aditya Chopra               Befikre          2016
5  Shubhashish Bhutiani       Hotel Salvation          2016
6         Mudassar Aziz     Happy Bhag Jayegi          2016
7          Remo D'Souza         A Flying Jatt          2016
8           Sohail Khan            Freaky Ali          2016
9          Umesh Ghadge  Kyaa Kool Hain Hum 3          2016
CPU times: user 73.8 ms, sys: 3.94 ms, total: 77.7 ms
Wall time: 81.9 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [6]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """   

SELECT 
      Name 
FROM   PERSON 
WHERE  PID IN (SELECT 
                      TRIM(PID) 
                 FROM M_Cast 
                WHERE MID 
                   IN (SELECT 
                              MID 
                         FROM Movie 
                        WHERE title='Anand' AND year = 1971))   
                 
                 """
grader_2(query2)

                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
CPU times: user 25.1 ms, sys: 0 ns, total: 25.1 ms
Wall time: 29.1 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [7]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
)
) r1 
on r1.PD=p.PID 
"""
query_more_1990 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
)
) r1 
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

(4942, 1)
(62570, 1)
True
CPU times: user 247 ms, sys: 16.1 ms, total: 263 ms
Wall time: 267 ms


In [8]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """ 
SELECT 
       Name 
  FROM Person 
WHERE PID IN
(SELECT 
        TRIM(PID) AS PD 
  FROM  M_Cast 
  WHERE  MID IN (SELECT 
                        MID 
                  FROM Movie 
                  WHERE CAST(SUBSTR(year,-4,4) AS INT)<1970)
INTERSECT
SELECT 
      TRIM(PID) AS PD 
  FROM M_Cast 
WHERE MID IN (SELECT 
                      MID
                FROM  Movie 
                WHERE  CAST(SUBSTR(year,-4,4) AS INT)>1990));
 """
grader_3(query3)

                Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
CPU times: user 135 ms, sys: 2.56 ms, total: 137 ms
Wall time: 142 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [9]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a ="""  

  SELECT
          TRIM(Person.Name) AS director_name,
          COUNT(*) AS no_of_movies_directed
    FROM  M_Director JOIN Person
      ON  M_Director.PID = Person.PID
GROUP BY  director_name;


 """
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

              director_name  no_of_movies_directed
0              A. Bhimsingh                      7
1            A. Jagannathan                      2
2                  A. Majid                      1
3                  A. Muthu                      1
4             A.M.R. Ramesh                      1
5            A.P. Nagarajan                      1
6           A.R. Murugadoss                      6
7  A.S. Ravi Kumar Chowdary                      2
8              Aamir Bashir                      1
9                Aamir Khan                      1
False
CPU times: user 64.8 ms, sys: 0 ns, total: 64.8 ms
Wall time: 67.6 ms


In [10]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """ 

  SELECT
          TRIM(Person.Name) AS director_name,
          COUNT(*) AS no_of_movies_directed
    FROM  M_Director JOIN Person
      ON  M_Director.PID = Person.PID
GROUP BY  director_name
  HAVING  no_of_movies_directed >=10
ORDER BY  no_of_movies_directed DESC;


 """
grader_4(query4)

          director_name  no_of_movies_directed
0          David Dhawan                     39
1          Mahesh Bhatt                     36
2          Priyadarshan                     30
3       Ram Gopal Varma                     30
4          Vikram Bhatt                     29
5  Hrishikesh Mukherjee                     27
6           Yash Chopra                     21
7       Basu Chatterjee                     19
8        Shakti Samanta                     19
9          Subhash Ghai                     18
CPU times: user 63.1 ms, sys: 3.09 ms, total: 66.2 ms
Wall time: 66 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [11]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa) 
    return (query_5aa.shape == (8846,3))
#  *** Write your query that will get movie id, and number of people for each geneder *** 
query_5aa ="""

  SELECT 
          M_Cast.MID,
          Person.Gender,
          COUNT(*) AS count 
    FROM  Person 
    JOIN  M_Cast 
      ON  Person.PID = TRIM(M_Cast.PID) 
GROUP BY  M_Cast.MID,Person.Gender;

"""

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab) 
    return (query_5ab.shape == (3469, 3))

# *** Write your query that will have at least one male actor try to use query that you have written above *** 
query_5ab =""" 

  SELECT 
          M_Cast.MID,
          Person.Gender,
          COUNT(*) AS count 
    FROM  Person 
    JOIN  M_Cast 
      ON  Person.PID = TRIM(M_Cast.PID) 
GROUP BY  M_Cast.MID,Person.Gender 
  HAVING  Person.Gender = 'Male' AND count >= 1;

"""

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question'''

            MID  Gender  count
0     tt0021594    None      1
1     tt0021594  Female      3
2     tt0021594    Male      5
3     tt0026274    None      2
4     tt0026274  Female     11
...         ...     ...    ...
8841  tt8932884  Female      1
8842  tt8932884    Male      2
8843  tt9007142    None      1
8844  tt9007142  Female      4
8845  tt9007142    Male      5

[8846 rows x 3 columns]
True
            MID Gender  count
0     tt0021594   Male      5
1     tt0026274   Male      9
2     tt0027256   Male      8
3     tt0028217   Male      7
4     tt0031580   Male     27
...         ...    ...    ...
3464  tt8581230   Male      5
3465  tt8698956   Male      5
3466  tt8852558   Male     14
3467  tt8932884   Male      2
3468  tt9007142   Male      5

[3469 rows x 3 columns]
True
CPU times: user 331 ms, sys: 9.83 ms, total: 341 ms
Wall time: 346 ms


In [12]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """ 
       
  SELECT 
        CAST(SUBSTR(TRIM(year),-4,4) AS INT) AS year,
        COUNT(*) AS Female_Cast_Only_Movies 
    FROM Movie 
  WHERE MID NOT IN 
                    (SELECT 
                            M_Cast.MID 
                      FROM  Person 
                      JOIN  M_Cast 
                        ON  Person.PID = TRIM(M_Cast.PID) 
                      WHERE  Person.Gender IN ('Male','None'))
GROUP BY year
ORDER BY year DESC; 

"""
grader_5a(query5a)

   year  Female_Cast_Only_Movies
0  2018                        1
1  2000                        1
2  1999                        1
3  1939                        1
CPU times: user 149 ms, sys: 3.07 ms, total: 152 ms
Wall time: 153 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [13]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """
SELECT 
        Y.year,
        (CAST(Y.Female_Cast_Only_Movies AS FLOAT) / CAST(X.total_movies_rel AS FLOAT)) AS Percentage_Female_Only_Movie,
        X.total_movies_rel AS Total_Movies

  FROM
(SELECT 
        CAST(SUBSTR(TRIM(year),-4,4) AS INT) AS years,
        COUNT(*) AS total_movies_rel
    FROM Movie
GROUP BY years
ORDER BY years
) AS X
INNER JOIN
(
SELECT 
        CAST(SUBSTR(TRIM(year),-4,4) AS INT) AS year,
        COUNT(*) AS Female_Cast_Only_Movies 
  FROM  Movie 
  WHERE  MID NOT IN 
                  (SELECT 
                          M_Cast.MID 
                      FROM Person 
                      JOIN M_Cast 
                        ON Person.PID = TRIM(M_Cast.PID) 
                    WHERE Person.Gender IN ('Male','None'))
  GROUP BY year
  ORDER BY year DESC 
) AS Y ON X.years = Y.year;

              
"""
grader_5b(query5b)

   year  Percentage_Female_Only_Movie  Total_Movies
0  1939                      0.500000             2
1  1999                      0.015152            66
2  2000                      0.015625            64
3  2018                      0.009615           104
CPU times: user 168 ms, sys: 4.06 ms, total: 172 ms
Wall time: 174 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: IF an actor played multiple roles, or IF it simply occurs multiple times in casts, we still count her/him only once.

In [14]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """ 

SELECT 
        Movie.title, 
        COUNT(DISTINCT M_Cast.PID) AS count
  FROM Movie 
  JOIN M_Cast 
    ON Movie.MID = M_Cast.MID
GROUP BY Movie.MID
ORDER BY count DESC;

 """
grader_6(query6)

                        title  count
0               Ocean's Eight    238
1                    Apaharan    233
2                        Gold    215
3             My Name Is Khan    213
4  Captain America: Civil War    191
5                    Geostorm    170
6                     Striker    165
7                        2012    154
8                      Pixels    144
9       Yamla Pagla Deewana 2    140
CPU times: user 184 ms, sys: 8.79 ms, total: 192 ms
Wall time: 192 ms


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [15]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))
#*** Write a query that computes number of movies in each year ***
query7a = """ 

SELECT 
        CAST(SUBSTR(TRIM(year),-4,4) AS INT) AS Movie_Year,
        COUNT(*) AS Total_Movies
    FROM Movie
GROUP BY Movie_Year;

"""
grader_7a(query7a)

# using the above query, you can write the answer to the given question

   Movie_Year  Total_Movies
0        1931             1
1        1936             3
2        1939             2
3        1941             1
4        1943             1
5        1946             2
6        1947             2
7        1948             3
8        1949             3
9        1950             2
CPU times: user 7.86 ms, sys: 0 ns, total: 7.86 ms
Wall time: 8.44 ms


In [16]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results)
    assert (q7b_results.shape == (713, 4))

    '''*** 
    Write a query that will do joining of the above table(7a) with itself 
    such that you will join with only rows IF the second tables year is <= current_year+9 and more than or equal current_year
    *** '''

query7b = """   

WITH data_table AS
            (SELECT 
                      CAST(SUBSTR(TRIM(year),-4,4) AS INT) AS Movie_Year,
                      COUNT(*) AS Total_Movies
                  FROM Movie
              GROUP BY Movie_Year)

SELECT 
        M1.Movie_Year,
        M1.Total_Movies,
        M2.Movie_Year,
        M2.Total_Movies 
  FROM data_table M1, data_table M2
 WHERE M1.Movie_Year <= M2.Movie_Year AND M2.Movie_Year <= M1.Movie_Year+9;

          """
grader_7b(query7b)
# IF you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

     Movie_Year  Total_Movies  Movie_Year  Total_Movies
0          1931             1        1931             1
1          1931             1        1936             3
2          1931             1        1939             2
3          1936             3        1936             3
4          1936             3        1939             2
..          ...           ...         ...           ...
708        2016           129        2017           126
709        2016           129        2018           104
710        2017           126        2017           126
711        2017           126        2018           104
712        2018           104        2018           104

[713 rows x 4 columns]
CPU times: user 12.9 ms, sys: 1.79 ms, total: 14.7 ms
Wall time: 16.9 ms


In [17]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results)
    assert (q7_results.shape == (1, 2))

query7 = """ 


SELECT 
      COUNT(*) AS Decade_Movie_Count, 
      CAST(dist_years AS VARCHAR) || '-' || CAST(dist_years+9 AS VARCHAR) AS Decade
FROM  (SELECT DISTINCT CAST(SUBSTR(TRIM(year),-4,4) AS INT) AS dist_years FROM Movie ORDER BY dist_years) AS distinct_years
JOIN  Movie 
  ON  year BETWEEN dist_years AND dist_years+9
GROUP BY dist_years+9
ORDER BY Decade_Movie_Count  DESC
LIMIT 1;

"""
grader_7(query7)
# IF you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

   Decade_Movie_Count     Decade
0                1126  2008-2017
CPU times: user 66.7 ms, sys: 1.86 ms, total: 68.6 ms
Wall time: 69.2 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [18]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

# *** Write a query that will results in number of movies actor-director worked together ***
query8a = """ 

SELECT 
        C.PID AS Actor_id,
        D.PID AS Director_id,
        COUNT(*) AS movies 
  FROM  M_Cast C 
  JOIN  M_Director D 
    ON  C.MID = D.MID 
GROUP BY Actor_id, Director_id;     
                
"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

     Actor_id Director_id  movies
0   nm0000002   nm0496746       1
1   nm0000027   nm0000180       1
2   nm0000039   nm0896533       1
3   nm0000042   nm0896533       1
4   nm0000047   nm0004292       1
5   nm0000073   nm0485943       1
6   nm0000076   nm0000229       1
7   nm0000092   nm0178997       1
8   nm0000093   nm0000269       1
9   nm0000096   nm0113819       1
CPU times: user 265 ms, sys: 19.9 ms, total: 285 ms
Wall time: 286 ms


In [19]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """ 
  
WITH main_table (actor_id,director_id,movies_done) AS

                (SELECT 
                        C.PID AS Actor_id,
                        D.PID AS Director_id,
                        COUNT(*) AS movies 
                  FROM  M_Cast C 
                  JOIN  M_Director D 
                    ON  C.MID = D.MID 
                GROUP BY Actor_id, Director_id),

      sub_table (sub_actor_id,max_movies_done) AS

               (SELECT 
                        actor_id,
                        MAX(movies_done)
                  FROM  main_table 
               GROUP BY actor_id)

SELECT 
        Name,
        movies_done 
  FROM PERSON
  JOIN
       (SELECT 
                actor_id,
                movies_done 
          FROM  main_table 
          JOIN sub_table 
            ON actor_id = sub_actor_id AND movies_done >= max_movies_done       
         WHERE director_id = (SELECT 
                                      PID 
                                FROM  Person 
                               WHERE  Name LIKE '%Yash_Chopra%')) AS X
ON PID = TRIM(actor_id);

"""
grader_8(query8)

               Name  movies_done
0     Shashi Kapoor            7
1       Yash Chopra            2
2    Akhtar-Ul-Iman            1
3         Murad Ali            1
4      Badri Prasad            1
5        Saira Banu            1
6        Raj Bharti            1
7     Ashwini Bhave            1
8   Andrew Bicknell            1
9    Paul Blackwell            1
(245, 2)
CPU times: user 424 ms, sys: 13.8 ms, total: 438 ms
Wall time: 439 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [20]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """ 




WITH Shahrukh_pid AS
                    (SELECT 
                            PID AS shahrukh_0_id 
                       FROM Person 
                      WHERE TRIM(Name) LIKE '%Shah_Rukh_Khan%'),
     Shahrukh_mid_table AS
                    (SELECT 
                            DISTINCT TRIM(m.MID) AS shahrukh_movies 
                       FROM M_Cast m, Shahrukh_pid s 
                      WHERE TRIM(m.PID) = s.shahrukh_0_id),

     Shahrukh_co_actors AS
                     (SELECT 
                              DISTINCT TRIM(PID) AS shahrukh_1
                        FROM  M_Cast,Shahrukh_mid_table m,Shahrukh_pid p 
                       WHERE  TRIM(MID) = m.shahrukh_movies AND TRIM(PID) != p.shahrukh_0_id)
SELECT 
        * 
  FROM Shahrukh_co_actors;




"""
grader_9a(query9a)


# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

  shahrukh_1
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 188 ms, sys: 3.93 ms, total: 192 ms
Wall time: 193 ms


In [21]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """

WITH Shahrukh_pid AS
                    (SELECT 
                            PID AS shahrukh_0_id 
                       FROM Person 
                      WHERE TRIM(Name) LIKE '%Shah_Rukh_Khan%'),
     Shahrukh_mid_table AS
                    (SELECT 
                            DISTINCT TRIM(m.MID) AS shahrukh_movies 
                       FROM M_Cast m, Shahrukh_pid s 
                      WHERE TRIM(m.PID) = s.shahrukh_0_id),

     Shahrukh_co_actors AS
                     (SELECT 
                              DISTINCT TRIM(PID) AS shahrukh_1
                        FROM  M_Cast,Shahrukh_mid_table m,Shahrukh_pid p 
                       WHERE  TRIM(MID) = m.shahrukh_movies AND TRIM(PID) != p.shahrukh_0_id),


      S2_MIDS AS
                    (SELECT 
                            DISTINCT TRIM(MID) AS S2_MID 
                       FROM M_Cast,Shahrukh_co_actors 
                      WHERE TRIM(PID) = shahrukh_1),
      S2_ACTORS AS
                    (SELECT 
                            DISTINCT TRIM(PID) AS S2_PID 
                       FROM M_Cast, S2_MIDS 
                      WHERE MID = S2_MID)

SELECT 
        Name 
  FROM Person 
 WHERE PID IN 
              (SELECT 
                      DISTINCT S2_PID
                 FROM S2_ACTORS 
                WHERE TRIM(S2_PID) NOT IN (SELECT 
                                                  *
                                             FROM  Shahrukh_co_actors)) AND Name NOT LIKE '%Shah_Rukh_Khan%';

"""
grader_9(query9)

                     Name
0            Freida Pinto
1             Rohan Chand
2            Damian Young
3         Waris Ahluwalia
4   Caroline Christl Long
5           Rajeev Pahuja
6       Michelle Santiago
7         Alicia Vikander
8            Dominic West
9          Walton Goggins
(25698, 1)
CPU times: user 666 ms, sys: 8.9 ms, total: 675 ms
Wall time: 677 ms
